In [ ]:
from deepface import DeepFace
from deepface.commons import distance as dst
from deepface.detectors import FaceDetector
import time
import cv2
from glob import glob as gl
import numpy as np
from tqdm import trange
from matplotlib import pyplot as plt

In [ ]:
detector_names = ['opencv','ssd','mtcnn', 'retinaface']
detector_name = 'ssd'
detector = FaceDetector.build_model(detector_name)

def face_detect(img):
    faces = FaceDetector.detect_face(detector, detector_name, img, align = False)
    return faces

In [ ]:
import tensorflow as tf
from yolov3_tf2.models import (
    YoloV3, YoloV3Tiny
)
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs, convert_output

num_classes = 1
tfrecord = None
weights = 'face-yolov3-tiny.tf'
classes = './yolov3_tf2/face.names'
size = 416
yolo = YoloV3Tiny(classes=num_classes)
yolo.load_weights(weights).expect_partial()


def face_detect(img_raw):
    img_orin = img_raw.copy()
    img_raw = tf.convert_to_tensor(img_raw, dtype=tf.uint8)
    img = tf.expand_dims(img_raw, 0)
    img = transform_images(img, size)
    boxes, scores, classes, nums = yolo(img)
    img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
    output = convert_output(img, (boxes, scores, classes, nums))
    outputs = []
    for out in output:
        top, right, bottom, left = out
        top = 0 if top <0 else top
        right = 0 if right <0 else right
        bottom = 0 if bottom <0 else bottom
        left = 0 if left <0 else left
        face = img_orin[top:bottom, left:right]
        outputs.append([face, [left, top, right, bottom]])
    return outputs

In [ ]:
def show(img):
    plt.imshow(img[:, :, ::-1])
    plt.show()

In [ ]:
img = cv2.imread("4197.jpg")
t1 = time.time()
faces = face_detect(img)
print(len(faces))
t2 = time.time()
print(t2-t1)
for face in faces:
    show(face[0])

In [ ]:
def test_model(model_name, detector_name, threshold, test_samples = 1000):
    user_folders = gl("users/*")
    model = DeepFace.build_model(model_name)
    detector = FaceDetector.build_model(detector_name)
    w1 = 0
    w2 = 0
    T = 0
    for i in trange(test_samples):
        users = np.random.randint(0, len(user_folders), 2)
        user1 = np.random.choice(gl(user_folders[users[0]]+"/*.jpg"))
        user2 = np.random.choice(gl(user_folders[users[1]]+"/*.jpg"))
        user1  = face_detect(cv2.imread(user1))[0][0]
        user2  = face_detect(cv2.imread(user2))[0][0]
        embedding1 = DeepFace.represent(user1, model = model, fast=True)
        embedding2 = DeepFace.represent(user2, model = model, fast=True)
        distance = dst.findEuclideanDistance(dst.l2_normalize(embedding1), dst.l2_normalize(embedding2))
        if users[0] == users[1] and distance > threshold :
            w1+=1
        if users[0] != users[1] and distance <= threshold :
            w2+=1
        T+=1
    print("Total:", T, w1, w2)
    print(w1*100/T)
    print(w2*100/T)
    print((w1+w2)*100/T)

In [ ]:
test_model("Dlib", 'ssd', 0.25, 100)

In [ ]:
test_model("VGG-Face", 'ssd', 0.6, test_samples  = 100)

In [ ]:
test_model("OpenFace", 'ssd', 0.5, test_samples  = 100)

In [ ]:
test_model("Facenet", 'ssd', 0.8, test_samples  = 100)

In [ ]:
test_model("ArcFace", 'ssd', 0.75, test_samples  = 100)